# 1. Data

## 1.1. Read Data

In [1]:
# Import the libraries 
import time
import random
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from collections import defaultdict
from queue import PriorityQueue

In [45]:
df = pd.read_csv("Distance.csv")
df.head()

,Province,Ho Chi Minh,Ha Noi,Hai Phong,Can Tho,Dong Nai,Quang Nam,Bac Ninh,Hai Duong,Nghe An,...,Ha Nam,Hung Yen,Kien Giang,Lai Chau,Da Nang,Dak Nong,Dien Bien,Vinh Phuc,Latitude,Longitude
0,Ho Chi Minh,0.000000,676.858073,651.191089,72.531672,17.825879,348.515470,677.163319,660.554247,516.655254,...,638.727039,655.764599,147.726583,780.839165,357.081088,115.136516,749.308296,690.958291,10.7638,106.6436
1,Ha Noi,676.858073,0.000000,73.725085,720.740193,667.911894,383.069620,34.707889,51.643740,160.339903,...,44.017361,36.130166,731.334185,154.401455,369.178207,590.387505,151.065337,14.674681,21.1382,105.5050
2,Hai Phong,651.191089,73.725085,0.000000,700.761317,640.677941,335.443495,43.665840,22.290550,150.274492,...,47.276965,39.548339,719.543196,226.546318,322.083230,556.899408,224.790404,76.915652,20.8000,106.6667
3,Can Tho,72.531672,720.740193,700.761317,0.000000,90.321287,412.019026,723.869751,708.489396,560.627540,...,684.596896,702.149911,82.484531,814.976141,419.497544,185.684427,781.701684,735.126667,10.0333,105.7833
4,Dong Nai,17.825879,667.911894,640.677941,90.321287,0.000000,334.205359,667.429155,650.493579,507.940916,...,629.274545,646.145822,165.025736,774.420059,343.117530,98.728234,743.390126,681.915838,10.9277,106.8681


## 1.2. Random Main Warehouse

In [4]:
# Random mainwarehouse: mw_north, mw_central, mw_south
# Create list store province bound -> random
list_mw_north, list_mw_central, list_mw_south = [], [], []
for i in range(df.shape[0]):
    if df.iloc[i,64] > 20:
        list_mw_north.append(df.iloc[i,0])
    elif df.iloc[i,64] < 12:
        list_mw_south.append(df.iloc[i,0])
    else: 
        list_mw_central.append(df.iloc[i,0])

# Random
random.seed(100)
mw_north, mw_central, mw_south = random.choice(list_mw_north), random.choice(list_mw_central), random.choice(list_mw_south)
print(mw_north,mw_central,mw_south, sep = "\n")

Nam Dinh
Phu Yen
Long An


## 1.3. Convert Distance -> Distance Truck -> Distance Plane (Main Warehouse)

In [5]:
# Convert distance -> cost
# Cost truck = (Distance / 60) * 35
def distance_to_truck_cost(dis_truck):
    return (dis_truck * 35) / 60 
# apply to convert distance to cost truck 
df.iloc[:, 1:64] = df.iloc[:, 1:64].apply(distance_to_truck_cost)

In [6]:
# Check data after convert distance to cost
df.iloc[:,:]

,Province,Ho Chi Minh,Ha Noi,Hai Phong,Can Tho,Dong Nai,Quang Nam,Bac Ninh,Hai Duong,Nghe An,...,Ha Nam,Hung Yen,Kien Giang,Lai Chau,Da Nang,Dak Nong,Dien Bien,Vinh Phuc,Latitude,Longitude
0,Ho Chi Minh,0.000000,394.833876,379.861469,42.310142,10.398430,203.300691,395.011936,385.323311,301.382232,...,372.590773,382.529349,86.173840,455.489513,208.297301,67.162968,437.096506,403.059003,10.763800,106.643600
1,Ha Noi,394.833876,0.000000,43.006299,420.431779,389.615271,223.457278,20.246269,30.125515,93.531610,...,25.676794,21.075930,426.611608,90.067515,215.353954,344.392711,88.121447,8.560231,21.138200,105.505000
2,Hai Phong,379.861469,43.006299,0.000000,408.777435,373.728799,195.675372,25.471740,13.002821,87.660120,...,27.578230,23.069865,419.733531,132.152019,187.881884,324.857988,131.127736,44.867464,20.800000,106.666700
3,Can Tho,42.310142,420.431779,408.777435,0.000000,52.687417,240.344432,422.257355,413.285481,327.032732,...,399.348189,409.587448,48.115976,475.402749,244.706900,108.315916,455.992649,428.823889,10.033300,105.783300
4,Dong Nai,10.398430,389.615271,373.728799,52.687417,0.000000,194.953126,389.333674,379.454588,296.298868,...,367.076818,376.918396,96.265013,451.745034,200.151893,57.591470,433.644240,397.784239,10.927700,106.868100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Lai Chau,455.489513,90.067515,132.152019,475.402749,451.745034,304.211114,106.972460,119.171893,163.333809,...,114.112013,111.141903,473.568928,0.000000,296.007756,412.657456,22.532261,87.297839,22.368662,103.311913
59,Da Nang,208.297301,215.353954,187.881884,244.706900,200.151893,8.203358,208.647956,196.722718,134.329647,...,189.966893,197.815116,269.716569,296.007756,0.000000,145.076044,283.881048,222.232326,16.054407,108.202164
60,Dak Nong,67.162968,344.392711,324.857988,108.315916,57.591470,139.301140,342.201418,331.602561,252.433598,...,320.748455,330.129226,145.956294,412.657456,145.076044,0.000000,396.020757,352.277781,12.264648,107.609802
61,Dien Bien,437.096506,88.121447,131.127736,455.992649,433.644240,292.067165,107.129234,118.220519,149.879658,...,109.564682,108.711679,453.072716,22.532261,283.881048,396.020757,0.000000,87.492378,21.804232,103.107651


In [7]:
# Find id mw
(id_north, id_central, id_south) = (0,0,0)
for i in range(63):
    for j in range(63):
        for k in range(63):
            if (df.iloc[i,0] == mw_north) & (df.iloc[j,0] == mw_central) & (df.iloc[k,0] == mw_south):
                id_north += i
                id_central += j
                id_south += k
                
# Cost plane = (Distance / 80) * 160 = (120 * Distance_Truck) / 35
# North
df.at[id_north, mw_central] = df.at[id_north, mw_central] * 120 / 35
df.at[id_north, mw_south] = df.at[id_north, mw_south] * 120 / 35
# Central
df.at[id_central, mw_north] = df.at[id_north, mw_north] * 120 / 35
df.at[id_central, mw_south] = df.at[id_north, mw_south] * 120 / 35
# South
df.at[id_south, mw_north] = df.at[id_north, mw_north] * 120 / 35
df.at[id_south, mw_central] = df.at[id_north, mw_central] * 120 / 35

In [8]:
# Check data after convert distance to cost
df.iloc[:,:]

,Province,Ho Chi Minh,Ha Noi,Hai Phong,Can Tho,Dong Nai,Quang Nam,Bac Ninh,Hai Duong,Nghe An,...,Ha Nam,Hung Yen,Kien Giang,Lai Chau,Da Nang,Dak Nong,Dien Bien,Vinh Phuc,Latitude,Longitude
0,Ho Chi Minh,0.000000,394.833876,379.861469,42.310142,10.398430,203.300691,395.011936,385.323311,301.382232,...,372.590773,382.529349,86.173840,455.489513,208.297301,67.162968,437.096506,403.059003,10.763800,106.643600
1,Ha Noi,394.833876,0.000000,43.006299,420.431779,389.615271,223.457278,20.246269,30.125515,93.531610,...,25.676794,21.075930,426.611608,90.067515,215.353954,344.392711,88.121447,8.560231,21.138200,105.505000
2,Hai Phong,379.861469,43.006299,0.000000,408.777435,373.728799,195.675372,25.471740,13.002821,87.660120,...,27.578230,23.069865,419.733531,132.152019,187.881884,324.857988,131.127736,44.867464,20.800000,106.666700
3,Can Tho,42.310142,420.431779,408.777435,0.000000,52.687417,240.344432,422.257355,413.285481,327.032732,...,399.348189,409.587448,48.115976,475.402749,244.706900,108.315916,455.992649,428.823889,10.033300,105.783300
4,Dong Nai,10.398430,389.615271,373.728799,52.687417,0.000000,194.953126,389.333674,379.454588,296.298868,...,367.076818,376.918396,96.265013,451.745034,200.151893,57.591470,433.644240,397.784239,10.927700,106.868100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Lai Chau,455.489513,90.067515,132.152019,475.402749,451.745034,304.211114,106.972460,119.171893,163.333809,...,114.112013,111.141903,473.568928,0.000000,296.007756,412.657456,22.532261,87.297839,22.368662,103.311913
59,Da Nang,208.297301,215.353954,187.881884,244.706900,200.151893,8.203358,208.647956,196.722718,134.329647,...,189.966893,197.815116,269.716569,296.007756,0.000000,145.076044,283.881048,222.232326,16.054407,108.202164
60,Dak Nong,67.162968,344.392711,324.857988,108.315916,57.591470,139.301140,342.201418,331.602561,252.433598,...,320.748455,330.129226,145.956294,412.657456,145.076044,0.000000,396.020757,352.277781,12.264648,107.609802
61,Dien Bien,437.096506,88.121447,131.127736,455.992649,433.644240,292.067165,107.129234,118.220519,149.879658,...,109.564682,108.711679,453.072716,22.532261,283.881048,396.020757,0.000000,87.492378,21.804232,103.107651


In [9]:
# Save new file with convert to (truck, plane) cost
df.to_csv('Cost.csv', index = False)

##  1.4. Add Main Warehouse (cost) to Data Another Main Warehouse (3)

In [50]:
def data_A_star(start,goal, mw_north, mw_central, mw_south):  
    # Data built in with Province 
    province_neighbor ={'Lai Chau':['Dien Bien','Lao Cai','Son La','Yen Bai'],
                'Yen Bai':['Lao Cai','Lai Chau','Son La','Ha Giang','Phu Tho','Tuyen Quang'],
                'Dien Bien':['Lai Chau','Son La'],
                'Thanh Hoa':['Son La','Nghe An','Ninh Binh','Hoa Binh'],
                'Nghe An':['Ha Tinh','Thanh Hoa'],
                'Quang Binh':['Ha Tinh','Quang Tri'] ,
                'Ha Tinh':['Nghe An','Quang Binh'],
                'Hue':['Quang Tri','Da Nang','Quang Nam'],
                'Quang Tri':['Quang Binh','Hue'],
                'Da Nang':['Hue','Quang Nam'],
                'Quang Nam':['Da Nang','Hue','Kon Tum','Quang Ngai'],
                'Kon Tum':['Quang Nam','Quang Ngai','Gia Lai'],
                'Quang Ngai':['Kon Tum','Quang Nam','Binh Dinh'],
                'Gia Lai':['Kon Tum','Binh Dinh','Phu Yen','Dak Lak'],
                'Binh Dinh':['Quang Ngai','Gia Lai','Phu Yen'],
                'Dak Lak':['Dak Nong','Lam Dong','Khanh Hoa','Phu Yen','Gia Lai'],
                'Phu Yen':['Gia Lai','Binh Dinh','Dak Lak','Khanh Hoa'],
                'Khanh Hoa':['Dak Lak','Phu Yen','Lam Dong','Ninh Thuan'],
                'Dak Nong':['Dak Lak','Lam Dong','Binh Phuoc'],
                'Lam Dong':['Dak Lak','Dak Nong','Binh Phuoc','Dong Nai','Binh Thuan','Ninh Thuan','Khanh Hoa'],
                'Ninh Thuan':['Lam Dong','Khanh Hoa','Binh Thuan'],
                'Binh Phuoc':['Tay Ninh','Binh Duong','Dong Nai','Dak Nong','Lam Dong'],
                'Dong Nai':['Lam Dong','Binh Phuoc','Binh Thuan','Ho Chi Minh','Vung Tau','Binh Duong'],
                'Binh Thuan':['Vung Tau','Lam Dong','Ninh Thuan','Dong Nai'],
                'Tay Ninh':['Long An', 'Ho Chi Minh', 'Binh Duong','Binh Phuoc'],
                'Binh Duong':['Tay Ninh','Ho Chi Minh','Dong Nai','Binh Phuoc'],
                'Ho Chi Minh':['Tay Ninh','Binh Duong','Dong Nai','Long An','Vung Tau'],
                'Vung Tau':['Ho Chi Minh','Dong Nai','Binh Thuan'],
                'Long An':['Tay Ninh','Ho Chi Minh','Dong Thap','Tien Giang'],
                'Dong Thap':['Long An','Tien Giang','Vinh Long','Can Tho','An Giang'],
                'Tien Giang':['Long An','Dong Thap','Vinh Long','Ben Tre'],
                'Ben Tre':['Vinh Long','Tien Giang','Tra Vinh'],
                'An Giang':['Kien Giang','Can Tho','Dong Thap'],
                'Can Tho':['Kien Giang','An Giang','Dong Thap','Vinh Long','Hau Giang'],
                'Vinh Long':['Can Tho','Dong Thap','Ben Tre','Tra Vinh','Hau Giang'],
                'Tra Vinh':['Ben Tre','Vinh Long','Soc Trang'],
                'Soc Trang':['Hau Giang','Bac Lieu','Tra Vinh'],
                'Hau Giang':[ 'Kien Giang','Bac Lieu','Soc Trang','Can Tho','Vinh Long'],
                'Kien Giang':['An Giang','Can Tho','Hau Giang','Bac Lieu','Ca Mau'],
                'Bac Lieu':['Kien Giang','Hau Giang','Soc Trang','Ca Mau'],
                'Ca Mau':['Kien Giang','Bac Lieu'],
                'Ha Noi':['Phu Tho','Hoa Binh','Bac Giang','Vinh Phuc','Bac Ninh','Hung Yen','Ha Nam'],
                'Phu Tho':['Son La','Yen Bai','Hoa Binh','Ha Noi','Vinh Phuc','Tuyen Quang'],
                'Hoa Binh':['Ninh Binh','Thanh Hoa','Ha Nam','Ha Noi','Phu Tho','Son La'],
                'Bac Giang':['Quang Ninh','Hai Duong','Bac Ninh','Ha Noi','Thai Nguyen','Lang Son'],
                'Vinh Phuc':['Phu Tho', 'Ha Noi','Thai Nguyen','Tuyen Quang'],
                'Bac Ninh':['Ha Noi','Hung Yen','Hai Duong', 'Bac Giang'],
                'Hung Yen':['Ha Noi','Bac Ninh','Hai Duong','Thai Binh','Ha Nam'],
                'Ha Nam':['Ha Noi','Hung Yen','Thai Binh','Nam Dinh','Ninh Binh','Hoa Binh'],
                'Hai Phong':['Thai Binh','Thai Binh','Quang Ninh'],
                'Hai Duong':['Bac Ninh', 'Hung Yen','Thai Binh','Hai Phong','Quang Ninh','Bac Giang'],
                'Nam Dinh':['Ninh Binh','Ha Nam','Thai Binh'],
                'Thai Nguyen':['Tuyen Quang', 'Bac Kan','Lang Son','Bac Giang','Ha Noi','Vinh Phuc'],
                'Thai Binh':['Nam Dinh', 'Ha Nam','Hung Yen','Hai Duong','Hai Duong','Hai Phong'],
                'Quang Ninh':['Hai Duong','Hai Phong','Bac Giang','Lang Son'],
                'Lang Son':['Quang Ninh','Bac Giang','Thai Nguyen','Bac Kan','Cao Bang'],
                'Ninh Binh':['Hoa Binh','Thanh Hoa','Nam Dinh','Ha Nam'],
                'Lao Cai':['Lai Chau','Yen Bai','Ha Giang'],
                'Cao Bang':['Ha Giang','Bac Kan','Lang Son'],
                'Ha Giang':['Cao Bang','Tuyen Quang','Yen Bai','Lao Cai'],
                'Tuyen Quang':['Ha Giang','Bac Kan','Thai Nguyen','Vinh Phuc','Phu Tho','Yen Bai'],
                'Bac Kan':['Tuyen Quang','Thai Nguyen','Lang Son','Cao Bang'],
                'Son La':['Dien Bien','Yen Bai','Phu Tho','Hoa Binh','Thanh Hoa'],
               }

    # Read Cost data
    df = pd.read_csv("Cost.csv")
    # Store name of each province 
    Province1 = df['Province']
    Province=[]
    for i in Province1:
        Province.append(i.strip())

    for name in Province: 
        name = name.strip()
    # Store Index for each province
    dict_province = {}
    for i in range(len(Province)):
        dict_province[Province[i]]=i
    
    # Store details cost t
    list_cost = []
    for i in df.values:
        list_cost.append(list(i[1:-2]))

    # Dict neighbor cost 
    dict_cost = {}
    for province in province_neighbor:
        # Create empty list to store name province + cost
        list_cost_province =[]
        for neighborhood in province_neighbor[province]:
            # append name neighborhood
            list_cost_province.append(neighborhood)
            # append cost to neigborhood
            list_cost_province.append(list_cost[dict_province[province]][dict_province[neighborhood]])

        # Add main ware house to data other main warehouse
        if province == mw_north:
            list_cost_province.append(mw_central)
            list_cost_province.append(list_cost[dict_province[province]][dict_province[mw_central]])
            list_cost_province.append(mw_south)
            list_cost_province.append(list_cost[dict_province[province]][dict_province[mw_south]])
        elif province == mw_central:
            list_cost_province.append(mw_north)
            list_cost_province.append(list_cost[dict_province[province]][dict_province[mw_north]])
            list_cost_province.append(mw_south)
            list_cost_province.append(list_cost[dict_province[province]][dict_province[mw_south]])
        elif province == mw_south:
            list_cost_province.append(mw_north)
            list_cost_province.append(list_cost[dict_province[province]][dict_province[mw_north]])
            list_cost_province.append(mw_central)
            list_cost_province.append(list_cost[dict_province[province]][dict_province[mw_central]])

        # Append cost from start to goal
        list_cost_province.append(list_cost[dict_province[province]][dict_province[goal]])
        # Assign list to dict
        dict_cost[province] = list_cost_province
        
    return dict_cost  
data = data_A_star("Nam Dinh", "Hue", mw_north, mw_central, mw_south)

['Tay Ninh',
 31.69395234055556,
 'Ho Chi Minh',
 12.49557809111111,
 'Dong Thap',
 28.82576592729167,
 'Tien Giang',
 7.185139496597222,
 'Nam Dinh',
 0.0,
 'Phu Yen',
 12088.605457371428,
 228.70532763263887]